# SLAP2 Pipeline Example
This document aims to demonstrate the SLAP2 pipelines highlighted in the paper. The SLAP2_Util library requires the installation of mandatory libraries, including CUPY, which relies on an NVIDIA GPU. Thus, it is highly recommended that the code be run on a computer with a GPU.
The overall pipeline is outlined in the following code block, where the data of each region of interest (ROI) is extracted and organized based on their index. The document will highlight each steps as well.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon_perimeter
from Desktop.SLAP2_Utils.slap2_utils.datafile import DataFile
from Desktop.SLAP2_Utils.slap2_utils.subclasses.metadata import MetaData
from Desktop.SLAP2_Utils.slap2_utils.utils.trace import Trace
from Desktop.SLAP2_Utils.slap2_utils.utils.roi_utils import roiBoolean, roiImg

hDataFile = DataFile('C://Users//Jerry//Desktop//testFile.dat');

raw_data = []
processed_data = []
print("Loading passed. Data extraction will start now.")

for _roi in range(len(hDataFile.metaData.AcquisitionContainer.ROIs)):
    roi_shape = hDataFile.metaData.AcquisitionContainer.ROIs[_roi].shapeData
    ROI = hDataFile.metaData.AcquisitionContainer.ROIs[_roi]
    zIdx = hDataFile.fastZs.index(ROI.z)
    chIdx = 1
    hTrace = Trace(hDataFile, zIdx, chIdx)

    roi_shape = roi_shape.astype(int)
    integrationPixels = roiBoolean(hDataFile, _roi)

    rasterPixels = np.full((800, 1280), False)
    hTrace.setPixelIdxs(rasterPixels, integrationPixels);
    individual_processed_trace, _, _, _ = hTrace.process(10, 1000)
    processed_data.append(individual_processed_trace)
    hTrace.orderadjust()
    individual_raw = []
    for x in range(len(hTrace.TracePixels)):
        individual_raw.append(hTrace.TracePixels[x].data[0][:400])
    raw_data.append(individual_raw)


print('Completed.')

Once the files are outputted from SLAP2, the DataFile object will be produced first. This object loads the binary files into the Python environment. When successful, the function will output "DataFile Created," which also indicates that the function has ended.

In [ ]:
hDataFile = DataFile('C://Users//Jerry//Desktop//testFile.dat');

If the user wish to view the ROI's shape in a 2D plane, the user will need to load the ROI shape from each ROI. The information can be extracted from skimage using the `polygon_perimeter` function or with roi_util functions. The `polygon_perimeter` function can only accept list with size larger than 2 such that it is actually a polygon, so extra loop must be implemented to account for this exception. The function utilizes matplotlib to show the image after.

In [ ]:
img2 = np.zeros((800, 1280), dtype=np.uint32)

for _roi in range(len(hDataFile.metaData.AcquisitionContainer.ROIs)):
    roi_shape = hDataFile.metaData.AcquisitionContainer.ROIs[_roi].shapeData
    if roi_shape.shape[0] == 2:
        if len(roi_shape[0,:]) > 2 :
            rr, cc = polygon_perimeter(roi_shape[0, :], roi_shape[1, :],
            shape=img2.shape, clip=True)
            img2[rr, cc] = _roi + 1
        else:
            rr = []
            cc = []
            for i in roi_shape[0, :]:
                for j in roi_shape[1, :]:
                    rr.append(int(i))
                    cc.append(int(j))
            img2[rr, cc] = _roi + 1

plt.imshow(img2)

If the user wish to know the ROI shape of a specific ROI (and the index that corresponds to it), user can use the `roiImg` function. 

In [ ]:
# If one wants to see ROI at the first index (0)
plt.plot(roiImg(hDataFile,0))

For the data to be extracted, the user must input a boolean array that indicates where the pixels are for specific ROIs. The channel index (chIdx) can be changed or extracted from the file. `Trace` is called, and the function `setPixelIdxs` is then called with the boolean array inputted. The `process` function loads the data into the object and outputs a convolved trace as `individual_processed_trace`. This trace is the trace result of the whole ROI, which is a summation of each individual superpixel inside a specific ROI.

In [ ]:
processed_data = []
print("Extracting processed trace...")

for _roi in range(len(hDataFile.metaData.AcquisitionContainer.ROIs)):
    roi_shape = hDataFile.metaData.AcquisitionContainer.ROIs[_roi].shapeData
    ROI = hDataFile.metaData.AcquisitionContainer.ROIs[_roi]
    zIdx = hDataFile.fastZs.index(ROI.z)
    chIdx = 1
    hTrace = Trace(hDataFile, zIdx, chIdx)

    roi_shape = roi_shape.astype(int)
    integrationPixels = roiBoolean(hDataFile, _roi)

    rasterPixels = np.full((800, 1280), False)
    hTrace.setPixelIdxs(rasterPixels, integrationPixels)
    
    individual_processed_trace, _, _, _ = hTrace.process(10, 1000)
    processed_data.append(individual_processed_trace)

print("Extraction completed.")

If the user wishes to extract the raw data inside the superpixels of the trace object (that also represents one ROI), the raw data is inside the `TracePixel` field. The function `orderadjust` must be called in this case because reordering in processed trace extraction is not necessary when it is outputting a sum of all superpixels. The example below extracts data of a specific range (the first superpixel, from 0 to the 400th acquisition), but the number is modifiable if the user wishes to sample a different range of data. The list here will include data from all the loops in the acquisition plan.

In [ ]:
raw_data = []
print("Extracting raw data...")

for _roi in range(len(hDataFile.metaData.AcquisitionContainer.ROIs)):
    roi_shape = hDataFile.metaData.AcquisitionContainer.ROIs[_roi].shapeData
    ROI = hDataFile.metaData.AcquisitionContainer.ROIs[_roi]
    zIdx = hDataFile.fastZs.index(ROI.z)
    chIdx = 1
    hTrace = Trace(hDataFile, zIdx, chIdx)

    roi_shape = roi_shape.astype(int)
    integrationPixels = roiBoolean(hDataFile, _roi)

    rasterPixels = np.full((800, 1280), False)
    hTrace.setPixelIdxs(rasterPixels, integrationPixels);
    hTrace.orderadjust()

    individual_processed_trace, _, _, _ = hTrace.process(10, 1000)
    processed_data.append(individual_processed_trace)
    hTrace.orderadjust()
    individual_raw = []
    for x in range(len(hTrace.TracePixels)):
        individual_raw.append(hTrace.TracePixels[x].data[0][:400])
    raw_data.append(individual_raw)

print("Extraction completed.")

The user can view the data through matplotlib (example below shows a plot of a raw and processed data at the first acquisition loop). Other libraries would work as well.

In [ ]:
processed_data_plot = plt.plot(processed_data[0])
plt.show()
raw_data_plot = plt.plot(raw_data[0][0])
plt.show()